In [7]:
import pandas as pd
df = pd.read_excel("DATA_Kiss_count_gender_and_IQ.xlsx")

In [8]:
print(df.columns)

Index(['Name', 'Gender', 'IQ', 'Kiss Count', 'Age of First Kiss', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Gender.1'],
      dtype='object')


In [13]:
import pandas as pd
from scipy.stats import pearsonr, t
from itertools import combinations

# 讀取 Excel 資料
df = pd.read_excel("DATA_Kiss_count_gender_and_IQ.xlsx")
df['Gender'] = df['Gender'].str.strip().str.lower()
df['GenderFlag'] = df['Gender'].apply(lambda x: 1 if x == 'male' else 0)
df = df.dropna(subset=['IQ', 'Kiss Count', 'Age of First Kiss', 'GenderFlag'])

# 样本數
n = len(df)

# 工具函數
def r_to_t(r, n):
    return r * ((n - 2) ** 0.5) / ((1 - r ** 2) ** 0.5)

def t_to_p(t_val, dfree):
    return 2 * t.sf(abs(t_val), dfree)

def significance(p):
    if p < 0.001:
        return "*** (p < 0.001)"
    elif p < 0.01:
        return "** (p < 0.01)"
    elif p < 0.05:
        return "* (p < 0.05)"
    else:
        return "n.s."

def sig_label(p):
    if not isinstance(p, (int, float)) or pd.isna(p):
        return "N/A"
    elif p < 0.05:
        return "Significant"
    else:
        return "Not sig."


# 欲分析的變數欄位
cols = ['IQ', 'Kiss Count', 'Age of First Kiss', 'GenderFlag']
results = []

# 計算所有兩兩變數的相關、t值、p值與顯著性
for var1, var2 in combinations(cols, 2):
    x = df[var1]
    y = df[var2]
    
    r_val, p_corr = pearsonr(x, y)
    t_val = r_to_t(r_val, n)
    p_from_t = t_to_p(t_val, n - 2)

    results.append({
        'Var A': var1,
        'Var B': var2,
        'Pearson r': round(r_val, 6),
        'Pearson p': round(p_corr, 6),
        't (from r)': round(t_val, 6),
        'p (from t)': round(p_from_t, 6),
        'Significance': significance(p_corr),
        'Label': sig_label(p_corr)  # ← 加這一欄
    })

# 為對稱矩陣補上相反順序
for var1, var2 in combinations(cols, 2):
    row = next(item for item in results if item['Var A'] == var1 and item['Var B'] == var2)
    results.append({
        'Var A': var2,
        'Var B': var1,
        'Pearson r': row['Pearson r'],
        'Pearson p': row['Pearson p'],
        't (from r)': row['t (from r)'],
        'p (from t)': row['p (from t)'],
        'Significance': row['Significance'],
        'Label': row['Label']
    })

# 結果輸出成 DataFrame
result_df = pd.DataFrame(results)
result_df = result_df.sort_values(by=['Var A', 'Var B'])
print(result_df)


                Var A              Var B  Pearson r  Pearson p  t (from r)  \
5   Age of First Kiss         GenderFlag  -0.194026   0.005906   -2.783069   
7   Age of First Kiss                 IQ  -0.041842   0.556346   -0.589279   
9   Age of First Kiss         Kiss Count  -0.230764   0.001011   -3.337208   
11         GenderFlag  Age of First Kiss  -0.194026   0.005906   -2.783069   
8          GenderFlag                 IQ   0.128213   0.070401    1.819135   
10         GenderFlag         Kiss Count  -0.003463   0.961185   -0.048728   
1                  IQ  Age of First Kiss  -0.041842   0.556346   -0.589279   
2                  IQ         GenderFlag   0.128213   0.070401    1.819135   
0                  IQ         Kiss Count   0.110812   0.118262    1.568927   
3          Kiss Count  Age of First Kiss  -0.230764   0.001011   -3.337208   
4          Kiss Count         GenderFlag  -0.003463   0.961185   -0.048728   
6          Kiss Count                 IQ   0.110812   0.118262  